In [ ]:
# import the required libraries 
import random 
import matplotlib.pyplot as plt 
    
    
slope     = 12
intercept = -3
alpha = 3
beta = 1
xvals =np.linspace(0.5, 5, 1000)# np.random.uniform(0.5, 5, size=80)
yvals = slope*xvals + intercept + np.random.gamma(alpha, beta, size = len(xvals))
gammaData = pd.DataFrame({'X' :xvals,
                          'Y' : yvals})


mod = smf.quantreg('Y ~ X', gammaData)



# Define the quantiles and fit:
quantiles  = np.linspace(0.01, 0.99, 100)
def fit_model(q):
    res = mod.fit(q=q)
    return [q, res.params['Intercept'], res.params['X']] + \
            res.conf_int().loc['X'].tolist()
    
models = [fit_model(x) for x in quantiles]
models = pd.DataFrame(models, columns=['q', 'a', 'b','lb','ub'])

ols = smf.ols('Y ~ X', gammaData).fit()
ols_ci = ols.conf_int().loc['X'].tolist()
ols = dict(a = ols.params['Intercept'],
           b = ols.params['X'],
           lb = ols_ci[0],
           ub = ols_ci[1])

print(models)
print(ols)
        
    
    
# Plotting the models:
xn = np.arange(min(xvals), max(xvals), 0.001)

get_y = lambda a, b: a + b * xn

fig, ax = plt.subplots(figsize=(16.18, 10))

for i in range(models.shape[0]):
    yn = get_y(models.a[i], models.b[i])
    ax.plot(xn, yn, linestyle='dotted', color='grey')

yn = get_y(ols['a'], ols['b'])



ax.scatter(xvals, yvals, alpha=.2, c='red')
legend = ax.legend()
ax.set_xlabel('X', fontsize=16)
ax.set_ylabel('Y', fontsize=16)
ax.set_title('Quantile regression with 0.05 and 0.95 quantiles')

fig.tight_layout()



